# Debug ConvLSTM 2D with history

In [ ]:
import os
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import yaml
import pickle as pkl
import subprocess
import datetime


# from keras.regularizers import L1L2
# from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Cropping2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import Reshape

from src.data_preparation import load_data
from src.data_preparation.blocking_time_series_split import BlockingTimeSeriesSplit
from src.data_preparation import mdl_dataset
from src.data_preparation import mdl_dataset_prep

from src.modelling import mdl_input_dico  # input variables class
from src.modelling import mdl_params  # parameters class
from src.modelling import mdl_history

# from src.modelling import model_autokeras
# from src.modelling import super_model_dl

from src.visualization import mdl_non_recursive
from src.visualization import mdl_ypred_PCA
from src.visualization import mdl_introspect

from src.utils import reload_config
from src.utils import tardisml_utils

from src.utils import save_name



def ds_2D_history(X, H):
    '''For dataset for Conv2D
    
    from shape : (1959, 329, 450, 8)  = (timeserie, width, length, features)
    to (1959, timesteps, 329, 450, 8) = (timeserie, timesteps=history, width, length, features)
    
    X : data for training
    H : history, array of time index to add
    
    '''
    
    ntime = X.shape[0]
    ydim = X.shape[1] 
    xdim = X.shape[2]
    nfeat = X.shape[-1]

    
    # If need different timesteps (History)
    needfutur, needpast = 0, 0
    if max(H)>0:
        needfutur = max(H)
    if min(H)<0:
        needpast = abs(min(H))
                
    # Number of data
    n = ntime - needpast - needfutur
    nH = len(H)
    
    X2 = np.empty([n, nH, ydim, xdim, nfeat])
    
    
    for t in range(n):
        for i, ts in enumerate(H):
            X2[t, i] = X[needpast+t+ts]
    
    
    return X2

def reshape_dim(xtrain, n_samples=50, timesteps=15, idx_samples=None):
    '''just for one feature for now
    
    # reshape (time serie, height, width) to (n_samples, timesteps, height, width, nfeature)
    3D to 5D

    Returns new array reshaped and random indexes used for sample selection
    
    n_samples      : int. number of sequence (batch?) of n frames (timesteps). number of samples
    timesteps      : int. number of frame in one sequence (temporal dimension)
    
    idx_samples    : array of indexes for sample selection. Default is None and random indexes are generated
    
    
    '''
    
    ntimes = xtrain.shape[0]
    height = xtrain.shape[1]
    width = xtrain.shape[2]
    
    new_data = np.zeros((n_samples, timesteps, height, width))
    if idx_samples is None:
        idx_samples = np.random.randint(0, ntimes-timesteps, size=n_samples)

    for ns in range(n_samples):
        new_data[ns] = xtrain[idx_samples[ns]:idx_samples[ns]+timesteps]
    
    # add one dimension for number of feature
    # (n_samples, timesteps, height, 1)
    return new_data[..., None], np.array(idx_samples)

def draw_loss(history, showfig=False, savefile=''):
    fig, ax1 = plt.subplots(1, 1, figsize=(12,4))

    epochs = range(len(history.history['loss']))
    
    ax1.plot(epochs, history.history['loss'], 'r', marker='.', label='Training')
    ax1.plot(epochs, history.history['val_loss'], 'b', linestyle="--", marker='.', label='Validation')

    ax1.set_ylabel('Loss')    
    ax1.set_xlabel('Epochs')

    
    plt.legend()
    plt.tight_layout()

    if savefile != '':
        odir = os.path.dirname(savefile)
        ofile = save_name.check(f"{odir}", os.path.basename(savefile))
        savefile = f'{odir}/{ofile}'
        plt.savefig(f"{savefile}")  # , facecolor='white')
        print(f'Saved as : {savefile}')

    if showfig:
        plt.show()

    plt.close() 
    

def scale_with_features(train_data, val_data, test_data):
    '''Features in last dimension
    Scale each feature independently of others
    
    Parameters:
    -----------
    
        train_data       :   array of dimension X, with nfeatures being the last dimension
    '''
    
    train_out = train_data.copy()
    val_out = val_data.copy()
    test_out = test_data.copy()
    
    for nfeat in range(train_data.shape[-1]):
        train_out[...,nfeat], val_out[...,nfeat], test_out[...,nfeat] = scale_data(train_data[...,nfeat],
                                                                                   val_data[...,nfeat],
                                                                                   test_data[...,nfeat]
                                                                                  )
    
    return train_out, val_out, test_out
    
    
def scale_data(train_data, val_data, test_data):
    '''Scale between 0 and 1
    
    Parameters:
    -----------
    
        train_data       :   array of dimension 1
    '''
    max_val = np.nanmax(train_data)  # .max()
    min_val = np.nanmin(train_data)  # .min()
    train_scaled = (train_data - min_val) / (max_val - min_val)
    val_scaled = (val_data - min_val) / (max_val - min_val)
    test_scaled = (test_data - min_val) / (max_val - min_val)
    return train_scaled, val_scaled, test_scaled

# ---------- MODEL --------------------------

def compile_ConvLSTM2d(nfeatures):
    '''
    '''
    
    seq = Sequential()

    seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                      input_shape=(None, 329, 450, nfeatures),
                       padding='same', return_sequences=True)) # , activation="relu"))
    seq.add(BatchNormalization())

    # 63
    seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                       padding='same', return_sequences=True))# , activation="relu"))
    seq.add(BatchNormalization())
    # 128
    seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                       padding='same', return_sequences=True))# , activation="relu"))
    seq.add(BatchNormalization())
    # 256
    seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                       padding='same', return_sequences=True)) #, activation="relu"))  # return_sequences=True
    seq.add(BatchNormalization())

    seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
                   activation='linear',
                   padding='same', data_format='channels_last'))
      # 'sigmoid',


    # seq.compile(loss='mse', optimizer=Adam(learning_rate=1e-4, clipnorm=1.))
    seq.compile(loss='mse', optimizer=Adam(learning_rate=1e-4))
    
    return seq


def compile_Conv2D(nfeatures):
    '''
    '''
    
    model = Sequential()
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), 
                     input_shape=(329, 450, nfeatures),
                     activation='relu', padding='same', data_format='channels_last'))

              
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu', padding='same'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
              
    model.add(Dense(1, activation='linear'))
              
    model.compile(loss='mse', optimizer=Adam(learning_rate=1e-4))
              
    return model

def compile_Conv2D_params(nfeatures):
    
    model = Sequential()
    
    model.add(Conv2D(filters=128, kernel_size=(5, 5), strides=(1,1), 
                     input_shape=(329, 450, nfeatures),
                     activation='relu', padding='same', data_format='channels_last'))

              
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu', padding='same'))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu', padding='same'))
              
    model.add(Dense(1, activation='linear'))
              
    model.compile(loss='mse', optimizer=Adam(learning_rate=1e-4))
              
    return model
    

def compile_Conv2D_encode_decode(nfeatures):
    
    model = Sequential()
    
    # Encoder
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(329, 450, nfeatures),
                     activation='relu', padding='same', data_format='channels_last'))
    model.add(MaxPooling2D((2,2), padding='same'))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2,2), padding='same'))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2,1), padding='same'))
    
    
    # Decoder
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2,2)))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2,2)))
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2,1)))
    
    model.add(Conv2D(filters=1, kernel_size=(3, 3), activation='relu', padding='same'))
    
    model.add(Cropping2D(cropping=((3, 4), (1, 1))))  # to correct output shape
    
    model.compile(loss='mse', optimizer=Adam(learning_rate=1e-4))
              
    return model

def compile_ConvLSTM2d_H(nfeatures, nH):
    '''
    format (timeserie, timesteps=history, width, length, features)
    
    
    Parameters:
    -----------
        nfeatures : int, number of features
        nH        : int, number of History timesteps
    
    '''
    
    seq = Sequential()

    # 32
    seq.add(ConvLSTM2D(filters=16, kernel_size=(3, 3),
                      input_shape=(nH, 329, 450, nfeatures),
                       padding='same', return_sequences=True,
                      go_backwards=True))    # , activation="relu"))
    seq.add(BatchNormalization())

    # 63
#     seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
#                        padding='same', return_sequences=True))# , activation="relu"))
#     seq.add(BatchNormalization())
#     # 128
#     seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
#                        padding='same', return_sequences=True))# , activation="relu"))
#     seq.add(BatchNormalization())
#     # 256
#     seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
#                        padding='same', return_sequences=True)) #, activation="relu"))  # return_sequences=True
#     seq.add(BatchNormalization())

    seq.add(MaxPooling3D(pool_size=(nH,1,1)))
    
    seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
                   activation='linear',
                   padding='same', data_format='channels_last'))
      # 'sigmoid',


    seq.add(Reshape((329,450)))
        
    # seq.compile(loss='mse', optimizer=Adam(learning_rate=1e-4, clipnorm=1.))
    seq.compile(loss='mse', optimizer=Adam(learning_rate=1e-4))
    
    return seq

# ------------------------------------------

In [3]:
file_config = '../config/config_to_jobs/stored/config_C2D_no_bias_0wk.yaml'

rootdir = tardisml_utils.get_rootdir()
conf = reload_config.Config(file_config, rootdir=rootdir, verbose=1)

# -----------------------

Config file found: ../config/config_to_jobs/stored/config_C2D_no_bias_0wk.yaml
PCA results in: /scratch/project_465000269/edelleo1/Leo/results/pca_i100-550_j300-629
Config file updated 'pca_dir': ../config/config_to_jobs/stored/config_C2D_no_bias_0wk.yaml
Results in: /scratch/project_465000269/edelleo1/Leo/results/C2D_230315-160727
Folder created

Subfolder created: /scratch/project_465000269/edelleo1/Leo/results/C2D_230315-160727/ml/
Subfolder created: /scratch/project_465000269/edelleo1/Leo/results/C2D_230315-160727/figures/
Config file updated 'results_dir': ../config/config_to_jobs/stored/config_C2D_no_bias_0wk.yaml
Config folders updated.
Config copied to: /scratch/project_465000269/edelleo1/Leo/results/C2D_230315-160727
Config file found: /scratch/project_465000269/edelleo1/Leo/results/C2D_230315-160727/config_C2D_no_bias_0wk.yaml
Default config file is now the copied following one:
/scratch/project_465000269/edelleo1/Leo/results/C2D_230315-160727/config_C2D_no_bias_0wk.yaml


In [4]:
# ---------------------------------------------------
#                 Loading data
# ---------------------------------------------------
print('Loading data...')
Xf, Xe, dsCo, dsFo, chrono, sia, mask = load_data.load_dataset_nc(file_config)

# data = (Xf, Xe, dsCo, dsFo, chrono)
Xe = Xe + Xf # trying with SIT assimilated instead of bias

# cap sea ice thickness at 0
# Xe = (Xe.where((0<Xe), 0)).where(np.isfinite(Xe))

# reverse latitude on sia
sia = sia.data[:, ::-1, :]
              
# ntrain, nval, ntest = mdl_dataset_prep.dataset_split(Xe.shape[0]) # , train_p=0.8, val_p=0.0)

# debug 
# ntrain = 100
# nval = 10
# ntest = 50

# backwards in time (TARDIS proof)
# Xtrain = Xf[ntest+nval:]
# ytrain = Xe[ntest+nval:]
# # ytest = Xe[100:150]  # [:ntest]

# Xval = Xf[ntest:ntest+nval]
# yval = Xe[ntest:ntest+nval]

# Xtest = Xf[:ntest]
# ytest = Xe[:ntest]

# ------------- RESHAPE: N SAMPLES, TIMESTEPS---------------------
# 5D
              
              
# n_samples_train = 200 # 500
# timesteps_train = 15

# n_samples_pred = 100  # 100
# timesteps_pred = 15

# xt, train_idx_samples = reshape_dim(Xtrain, n_samples_train, timesteps_train)
# yt, _ = reshape_dim(ytrain, n_samples_train, timesteps_train, idx_samples=train_idx_samples)

# test_idx_samples = np.arange(0, ntest-timesteps_pred)
# xttest, _ = reshape_dim(Xtest, n_samples_pred, timesteps_pred, idx_samples=test_idx_samples)
# yttest, _ = reshape_dim(ytest, n_samples_pred, timesteps_pred, idx_samples=test_idx_samples)


# ------------- RESHAPE ---------------------
# shape (batch_size, time_steps, height, width, layers)

# xxt = Xtrain.values.reshape((Xtrain.shape[0], -1, 329, 450))
# xt = xxt.reshape((Xtrain.shape[0], 1, 329, 450, -1))

# yyt = ytrain.values.reshape((ytrain.shape[0], -1, 329, 450))
# yt = yyt.reshape((ytrain.shape[0], 1, 329, 450, -1))

# xtest = Xtest.values.reshape((Xtest.shape[0], -1, 329, 450))
# xttest = xtest.reshape((Xtest.shape[0], 1, 329, 450, -1))

# yytest = ytest.values.reshape((ytest.shape[0], -1, 329, 450))
# yttest = yytest.reshape((ytest.shape[0], 1, 329, 450, -1))


              
              

# ------------- ADD VARIABLE 5D --------------------

# add covariable or forcing to dataset

# f2t, _ = reshape_dim(dsFo['2T_mean29d'], n_samples_train, timesteps_train, idx_samples=train_idx_samples)
# fmsl, _ = reshape_dim(dsFo['MSL_mean29d'], n_samples_train, timesteps_train, idx_samples=train_idx_samples)

# snth, _ = reshape_dim(dsCo['sisnthick'], n_samples_train, timesteps_train, idx_samples=train_idx_samples)
# sicc, _ = reshape_dim(dsCo['siconc'], n_samples_train, timesteps_train, idx_samples=train_idx_samples)

# Xtrain_in = np.concatenate((xt, snth, sicc), axis=4)  # stack on already existing axis 
# nfeatures = Xtrain_in.shape[-1]


# # f2t_test, _ = reshape_dim(dsFo['2T_mean29d'], n_samples_pred, timesteps_pred, idx_samples=test_idx_samples)
# # msl_test, _ = reshape_dim(dsFo['MSL_mean29d'], n_samples_pred, timesteps_pred, idx_samples=test_idx_samples)

# snth_test, _ = reshape_dim(dsCo['sisnthick'], n_samples_pred, timesteps_pred, idx_samples=test_idx_samples)
# sicc_test, _ = reshape_dim(dsCo['siconc'], n_samples_pred, timesteps_pred, idx_samples=test_idx_samples)

# Xtest_in = np.concatenate((xttest, snth_test, sicc_test), axis=4)


# ------------ import SIA -------------------------

# filename = os.path.join('/scratch/project_465000269/edelleo1/', 'Leo/sia/', 'Topaz_arctic25km_sea_ice_age_v2p0_20111001_20191231.nc')
# sia, chrono_sa = load_nc(f'{filename}', 'sia', X_only=True)



# import pdb; pdb.set_trace()
# -------------------- Build full dataset --------------------
# 4D
# Xtrain_in = np.stack((Xtrain, dsCo['sisnthick'][ntest+nval:], dsCo['siconc'][ntest+nval:],
#                      dsFo['2T_mean29d'][ntest+nval:], dsFo['MSL_mean29d'][ntest+nval:],
# #                      sia[ntest+nval:],
#                      np.repeat(mask.data[None,...], ntrain, axis=0) 
#                      ), axis=-1)  
# # stack on new axis
# Xval_in = np.stack((Xval, dsCo['sisnthick'][ntest:ntest+nval], dsCo['siconc'][ntest:ntest+nval],
#                    dsFo['2T_mean29d'][ntest:ntest+nval], dsFo['MSL_mean29d'][ntest:ntest+nval],
# #                    sia[ntest:ntest+nval],
#                    np.repeat(mask.data[None,...], nval, axis=0)
#                    ), axis=-1)

# Xtest_in = np.stack((Xtest, dsCo['sisnthick'][:ntest], dsCo['siconc'][:ntest],
#                     dsFo['2T_mean29d'][:ntest], dsFo['MSL_mean29d'][:ntest], 
# #                     sia[:ntest],
#                     np.repeat(mask.data[None,...], ntest, axis=0)  
#                     ), axis=-1)

# nfeatures = Xtrain_in.shape[-1]

# # inputs = ['Xf', 'sisnthick', 'siconc', '2T', 'MLS', 'SIA', 'lat', 'lon']
# inputs = ['Xf', 'sisnthick', 'siconc', '2T', 'MLS', 'mask land/ocean']
# print(inputs)
# add mask land/ocean to see if difference

# ------------- ADD HISTORY ---------------------
# build 4D dataset (timeserie, width, length, features)
# to convert it to 5D (timeserie, time steps=history, width, lenght, features)

X_new = np.stack((Xf, dsCo['sisnthick'], dsCo['siconc'],
                     dsFo['2T_mean29d'], dsFo['MSL_mean29d'],
#                      sia[ntest+nval:],
                     np.repeat(mask.data[None,...], Xf.shape[0], axis=0) 
                     ), axis=-1)  

nfeatures = X_new.shape[-1]

# inputs = ['Xf', 'sisnthick', 'siconc', '2T', 'MLS', 'SIA', 'lat', 'lon']
inputs = ['Xf', 'sisnthick', 'siconc', '2T', 'MLS', 'mask land/ocean']
print(inputs)

# -------------------------------

H = [0, 8, 16, 24]
needfutur = max(H)

X_in = ds_2D_history(X_new, H)
# Xtrain_in = ds_2D_history(Xtrain_in, H)
# Xval_in = ds_2D_history(Xval_in, H)
# Xtest_in = ds_2D_history(Xtest_in, H)

y_in = Xe[:-max(H)]
# ytrain = ytrain[:-max(H)]
# yval = yval[:-max(H)]
# ytest = ytest[:-max(H)]

chrono = chrono[:-max(H)]
# need to modify 'chrono'


# Splitting dataset
ntrain, nval, ntest = mdl_dataset_prep.dataset_split(X_in.shape[0]) # , train_p=0.8, val_p=0.0)


Xtrain_in = X_in[ntest+nval:]
Xval_in = X_in[ntest:ntest+nval]
Xtest_in = X_in[:ntest]

ytrain = y_in[ntest+nval:]
yval = y_in[ntest:ntest+nval]
ytest = y_in[:ntest]

Loading data...
Loading sia...
['Xf', 'sisnthick', 'siconc', '2T', 'MLS', 'mask land/ocean']


In [ ]:
# check xtrain, xval, xtest have correct chronological order

In [10]:
ctrain = chrono[ntest+nval:]
cval = chrono[ntest:ntest+nval]
ctest = chrono[:ntest]

In [13]:
ctest

cval

ctrain

,date
0,2011-10-01
1,2011-10-02
2,2011-10-03
3,2011-10-04
4,2011-10-05
...,...
594,2013-05-17
595,2013-05-18
596,2013-05-19
597,2013-05-20


In [5]:
Xtrain_in.shape

(1943, 4, 329, 450, 6)

In [15]:
ntest, nval, ntrain

(599, 448, 1943)

In [18]:
y_in.shape

(2990, 329, 450)

In [20]:
              
ypred_xr = xr.DataArray(data=y_in[:,:,:], 
                        dims=['time','y','x'], 
                        coords={'time':chrono[:].squeeze(),'y':Xf.y,'x':Xf.x},
                        name='sit_pred')

ytrue_xr = xr.DataArray(data=y_in[:,:,:], 
                        dims=['time','y','x'], 
                        coords={'time':chrono[:].squeeze(),'y':Xf.y,'x':Xf.x},
                        name='sit_pred')

ods = xr.Dataset(data_vars={'ypred':ypred_xr,
                            'ytrue':ytrue_xr},
                           attrs=dict(
                               description='Prediction values for SIT between TOPAZ4b and TOPAZ4b FreeRun',
                               model_ml=f'ConvLSTM2D',
                               author='Leo Edel, Nersc',
                               project='TARDIS',
                               date=f'{datetime.date.today()}')
                          )

In [21]:
ods

<xarray.Dataset>
Dimensions:  (time: 2990, y: 329, x: 450)
Coordinates:
  * time     (time) datetime64[ns] 2011-10-01 2011-10-02 ... 2019-12-07
  * y        (y) float32 -17.5 -17.38 -17.25 -17.12 ... 23.12 23.25 23.38 23.5
  * x        (x) float32 -25.5 -25.38 -25.25 -25.12 ... 30.25 30.38 30.5 30.62
Data variables:
    ypred    (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    ytrue    (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    description:  Prediction values for SIT between TOPAZ4b and TOPAZ4b FreeRun
    model_ml:     ConvLSTM2D
    author:       Leo Edel, Nersc
    project:      TARDIS
    date:         2023-03-15